### Choosing a metric

I first looked at class ditribution. They are binary classes, and the dataset is highly imbalanced because the true class makes up only ~0.19% of all labels.

I suspected if I just run a simple logistic regression on the dataset, I could achieve 99.81% accuracy, because the model can simply cheat by classifying every sample as 0. I ran logistic regression using sklearn with default params, and the result confirmed my hypothesis - the model predicted every sample as 0.

Obviously, accuracy is a bad metric for such an imbalanced dataset, especially in the RTB context, because being able to find samples that likely will result in a click-through is more important than overall correctness. If the model predicts the majority label for every sample, precision will be perfect but recall will be 0. If the model predicts the minority label for every sample, the recall will be perfect but precision will be 1. One potential metric I can use is the f1 score, but it states that precision and recall are equally important, which in reality may not be true. If capturing most of the click-throughs are more important than serving efficiency (most of the served impressions result in click-throughs, in other words, there is no infrastructure bottlenck), then recall is more important than precision. On the other hand, if serving efficiency is more important than capturing most of click-throughs, which may be true when the scale is large, then precision is more important. But in either case, f1 score can be very low even if we are happy with the model.

Without making too much an assumption of what we would want in real life, ROC AUC is a better metric to use because it depicts the tradeoff between specificity and senstivity. A large ROC AUC value indicates the model is more robust, and the optimal value should approach 1.

### Baseline models

#### Logistic Regression
I reran the default logistic regression model and print ROC AUC, f1 score and the confusion matrix. I use the binary labels as predicted values rather than scores, because I didn't want ROC AUC to be skewed by precision, and ultimatly I care more about the repdiced labels than probablities
```
================================
Confusion Matrix:
True Negative = 199623
False Negative = 377
True Positive = 0
False Positive = 0
================================
f1 score = 0.000
================================
ROC AUC Score = 0.500
```
F1 score is coerced to 0 because there are no predicted positives at all. ROC AUC score is low - just as good as a random classifier. Just for fun, I added l2 regularization `penalty='l2'`, the result is the same. The imbalance is just to strong for the model to even try to predict positives.

#### Neural Net
The baseline neural net has 1 hidden layer that outputs 88 weights for all features, and an output layer with 2 outputs, one probability for each class, with a softmax activation. Categorical crossentropy is used as loss.

The model is fit with 3 epochs, and a batch size of 32. The baseline neural net should be equivalent of a logisc regression, except that I'm using the SGD optimizer, so I picked a learning rate of 0.01 (it usually worked well for me when I train baseline models). I specifically set the inital weights as `Ones`, because Keras defaults the inital weights to `glorot_uniform`, which generally works really well (and too well for a baseline model).

I used set split to 0.2 for validation after each epoch.
Both training and validation error went down, and validation error is slightly smaller than training error in all epochs, indicating there is no overfitting. 

Unsurprisingly, the accuracy is still 99.81%, with the vast majority of samples predicted negative. ROC AUC score is still 0.5.

The result:
```
================================
Confusion Matrix:
True Negative = 199619
False Negative = 377
True Positive = 0
False Positive = 4
================================
f1 score = 0.000
================================
ROC AUC Score = 0.500
```

Because I set the initial weights to `Ones`, I wanted to eliminate the possibility that my weights are exploding or diminished. After checking the hidden layer's output weights, they are all between -1 and 1. So my baseline neural net is fine, even if I remove my initial weights so that they are set to `glorot_uniform`, the result is similar, there is one true positive and more false positives
```
================================
Confusion Matrix:
True Negative = 199418
False Negative = 376
True Positive = 1
False Positive = 205
================================
f1 score = 0.003
================================
ROC AUC Score = 0.501
```

### Model improvements
I think the main obstacle to achieving higher ROC AUC is the severe data imbalance. There are a few potential migations I can try:

- Deepen the neural network model
  <p>
    If add more layers, the model will have a chance to explore nonlinearty, which might cover the positive space.
  </p>
- Down sampling
  <p>
    The negative class can be overpowering. We could find a good class ratio so that the model can be trained for positive classes without underfitting the negative class. A side benefit is the model will take less time to train.
  </p>
- Over sampling
  <p>
    Like down sampling, the goal is to make the data more balanced. If we observe underfitting for the negative class, we can try oversampling the positive class and find a good class ratio. Oversampling can be done with random selection and duplication. This could cause overfitting for the positive class, and definitely slows down training and can increase model size.
  </p>
- SMOTE
  <p>
    We can also try a hybrid approach, down sampling negative class while generating synthetic samples for the positive class eith nearest neighbours. This could help prevent positive class overfitting. But just like oversampling, it causes the model to train slower and the size to increase.
  </p>
  
    



In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.ensemble import BalanceCascade, EasyEnsemble
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.model_selection import GridSearchCV
import h5py
import keras
from sklearn.utils import class_weight
from keras.utils import to_categorical
from keras.optimizers import *
from keras.regularizers import *
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier


%matplotlib inline

KFOLD_SEED = 42


def rtb_confusion_matrix(test_labels, y_preds):
    m = confusion_matrix(test_labels, y_preds)
    
    print("================================")
    print("Confusion Matrix:")
    print("True Negative = %d" % m[0][0])
    print("False Negative = %d" % m[1][0])
    print("True Positive = %d" % m[1][1])
    print("False Positive = %d" % m[0][1])


def rtb_f1_score(test_labels, y_preds):
    f = f1_score(test_labels, y_preds)
    print("================================")
    print("f1 score = %0.3f" % f)


def print_metrics(true_labels, y_preds, y_scores, is_train=True):
    if is_train:
        print("--------train---------")
    else:
        print("--------test---------")
    
    rtb_confusion_matrix(true_labels, y_preds)
    rtb_f1_score(true_labels, y_preds)
    print("================================")
    print("ROC AUC Score = %0.3f" % roc_auc_score(true_labels, y_scores.argmax(axis=-1)))
    
def keras_print_metrics(true_labels, y_scores, is_train=True):
    y_preds = y_scores.argmax(axis=-1)
    
    if is_train:
        print("--------train---------")
    else:
        print("--------test---------")
    
    rtb_confusion_matrix(true_labels, y_preds)
    rtb_f1_score(true_labels, y_preds)
    print("================================")
    print("ROC AUC Score = %0.3f" % roc_auc_score(true_labels, y_preds))

In [11]:
input_path = '~/data/biddings.csv'
data = pd.read_csv(input_path)
print(data.shape)

train = data[:800000]
test = data[800000:]

sample = train.sample(frac=1)
features = sample.drop('convert', axis=1).values
labels = sample.convert.ravel()
categorical_labels = to_categorical(labels, 2)

test_features = test.drop('convert', axis=1).values
test_labels = test.convert.ravel()
categorical_test_labels = to_categorical(test_labels, 2)

(1000000, 89)


### Baseline models

In [9]:
lr = LogisticRegression(penalty='l2', random_state=KFOLD_SEED, verbose=2)

model = lr.fit(features, labels)
predicted_scores = model.predict_proba(test_features)
predicted_labels = model.predict(test_features)
print(predicted_scores.shape, predicted_labels.shape)

print_metrics(test_labels, predicted_labels, predicted_scores, is_train=False)


[LibLinear](200000, 2) (200000,)
--------test---------
Confusion Matrix:
True Negative = 199623
False Negative = 377
True Positive = 0
False Positive = 0
f1 score = 0.000
ROC AUC Score = 0.500


/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
model = Sequential()
model.add(Dense(88, input_shape=(88,)))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])

history = model.fit(features, categorical_labels,
                    batch_size=32,
                    epochs=3,
                    callbacks=[keras.callbacks.EarlyStopping()],
                    validation_split=0.2,
                    verbose=1)

y_scores = model.predict(test_features, verbose=1)

keras_print_metrics(test_labels, y_scores)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 88)                7832      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 178       
Total params: 8,010
Trainable params: 8,010
Non-trainable params: 0
_________________________________________________________________
Train on 640000 samples, validate on 160000 samples
Epoch 1/3
640000/640000 [==============================] - 39s 61us/step - loss: 0.0321 - acc: 0.9964 - val_loss: 0.0300 - val_acc: 0.9970
Epoch 2/3
640000/640000 [==============================] - 39s 61us/step - loss: 0.0279 - acc: 0.9972 - val_loss: 0.0300 - val_acc: 0.9970
Epoch 3/3
200000/200000 [==============================] - 4s 19us/step
--------train---------
Confusion Matrix:
True Negative = 199418
False Negative = 376
True Positive = 1
False Positive = 205
f1 score =

In [20]:
w = model.get_layer(name='dense_10').get_weights()

In [3]:
def logistic_regression():
    lr = LogisticRegression(penalty='l2', random_state=KFOLD_SEED, verbose=2)
    return lr
    

def ensembler_test(classifier_fn, ensemblers):
    rus = RandomUnderSampler(ratio={0: 1531*10, 1: 1531}, random_state=KFOLD_SEED)
    X_us, y_us = rus.fit_sample(features, labels)
    
    for i, e in enumerate(ensemblers):
        print("fitting sample")
        X_res, y_res = e.fit_sample(X_us, y_us)
        print(X_res.shape, y_res.shape)
        clf = classifier_fn()
        print("training")
        
        for j, X_train in enumerate(X_res):
            model = clf.fit(X_train, y_res[j])
        
        predicted_scores = model.predict_proba(test_features)
        predicted_labels = model.predict(test_features)
        
        print("Ensembler %d" % i)
        print_metrics(test_labels, predicted_labels, predicted_scores, is_train=False)

EasyEnsemble and decision tree are consistenly the best

In [4]:
ee = EasyEnsemble(random_state=KFOLD_SEED)
bc = BalanceCascade(random_state=KFOLD_SEED)

dt = DecisionTreeClassifier(random_state=KFOLD_SEED)
bc_dt = BalanceCascade(estimator=dt, random_state=KFOLD_SEED)

rf = RandomForestClassifier(n_jobs=-1, random_state=KFOLD_SEED, verbose=1)
bc_rf = BalanceCascade(estimator=rf, random_state=KFOLD_SEED)

xgbc = XGBClassifier()
bc_xgbc = BalanceCascade(estimator=xgbc, random_state=KFOLD_SEED)


ensemblers = [bc_xgbc, ee, bc, bc_dt, bc_rf]
ensembler_test(logistic_regression, ensemblers)

fitting sample


/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/pr

/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/.local/share/virtualenvs/RTB-V2Lvgo6A/lib/python3.5/site-packages/sklearn/pr

(17, 3062, 88) (17, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 102476
False Negative = 162
True Positive = 215
False Positive = 97147
f1 score = 0.004
ROC AUC Score = 0.542
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 1
--------test---------
Confusion Matrix:
True Negative = 126466
False Negative = 115
True Positive = 262
False Positive = 73157
f1 score = 0.007
ROC AUC Score = 0.664
fitting sample
(19, 3062, 88) (19, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ens

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_j

(15, 3062, 88) (15, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 4
--------test---------
Confusion Matrix:
True Negative = 130478
False Negative = 145
True Positive = 232
False Positive = 69145
f1 score = 0.007
ROC AUC Score = 0.635


Try to tune EasyEnsemble by adjusting subsets. It does not affect f1-score or ROC AUC socre
```
ee = EasyEnsemble(n_subsets = 100, random_state=KFOLD_SEED)
ensembler_test(logistic_regression, [ee])

ee = EasyEnsemble(n_subsets = 4, random_state=KFOLD_SEED)
ensembler_test(logistic_regression, [ee])
```
Both result in:
```
================================
Confusion Matrix:
True Negative = 127470
False Negative = 132
True Positive = 245
False Positive = 72153
================================
f1 score = 0.007
================================
ROC AUC Score = 0.644
```

Tune max subset for BalanceCascade with DecisionTreeClassifier

10 is the best

In [19]:
dt_20 = DecisionTreeClassifier(random_state=KFOLD_SEED)
bc_dt_20 = BalanceCascade(estimator=dt_20, n_max_subset=20, random_state=KFOLD_SEED)

dt_10 = DecisionTreeClassifier(random_state=KFOLD_SEED)
bc_dt_10 = BalanceCascade(estimator=dt_10, n_max_subset=10, random_state=KFOLD_SEED)

dt_5 = DecisionTreeClassifier(random_state=KFOLD_SEED)
bc_dt_5 = BalanceCascade(estimator=dt_5, n_max_subset=5, random_state=KFOLD_SEED)

ensemblers = [bc_dt_20, bc_dt_10, bc_dt_5]
ensembler_test(logistic_regression, ensemblers)

fitting sample
(17, 3062, 88) (17, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 134778
False Negative = 150
True Positive = 227
False Positive = 64845
f1 score = 0.007
ROC AUC Score = 0.639
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 1
--------test---------
Confusion Matrix:
True Negative = 131774
False Negative = 133
True Positive = 244
False Positive = 67849
f1 score = 0.007
ROC AUC Score = 0.654
fitting sample
(5, 3062, 88) (5, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 2
--------test---------
Confusion Matrix:
True Negative = 128482
False Negative = 138
True Positive = 239
False Positive = 71141
f1 sc

Tune max features, overall it doesn't affect ROC AUC much, but 17 features is slightly higher than others

In [25]:
# 70 features
dt_08 = DecisionTreeClassifier(max_features=0.8, random_state=KFOLD_SEED)
bc_dt_08 = BalanceCascade(estimator=dt_08, n_max_subset=10, random_state=KFOLD_SEED)

# 35 features
dt_04 = DecisionTreeClassifier(max_features=0.4, random_state=KFOLD_SEED)
bc_dt_04 = BalanceCascade(estimator=dt_04, n_max_subset=10, random_state=KFOLD_SEED)

# 17 features
dt_02 = DecisionTreeClassifier(max_features=0.2, random_state=KFOLD_SEED)
bc_dt_02 = BalanceCascade(estimator=dt_02, n_max_subset=10, random_state=KFOLD_SEED)

# Auto is sqrt(n_features) ~= 9
dt_auto = DecisionTreeClassifier(max_features='auto', random_state=KFOLD_SEED)
bc_dt_auto = BalanceCascade(estimator=dt_auto, n_max_subset=10, random_state=KFOLD_SEED)

ensemblers = [bc_dt_08, bc_dt_04, bc_dt_02, bc_dt_auto]
ensembler_test(logistic_regression, ensemblers)

fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131814
False Negative = 132
True Positive = 245
False Positive = 67809
f1 score = 0.007
ROC AUC Score = 0.655
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 1
--------test---------
Confusion Matrix:
True Negative = 133049
False Negative = 142
True Positive = 235
False Positive = 66574
f1 score = 0.007
ROC AUC Score = 0.645
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 2
--------test---------
Confusion Matrix:
True Negative = 131934
False Negative = 126
True Positive = 251
False Positive = 67689
f1 score = 0.007
ROC AUC 

No need to tune decision tree's class weight, since ensembler already ensures both classes have equal samples

Min samples at leaves do not seem to matter

In [30]:
dt_min_samples_50 = DecisionTreeClassifier(min_samples_leaf=50, max_features=0.2, random_state=KFOLD_SEED)
bc_dt_min_samples_50 = BalanceCascade(estimator=dt_min_samples_50, n_max_subset=10, random_state=KFOLD_SEED)

dt_min_samples_20 = DecisionTreeClassifier(min_samples_leaf=20, max_features=0.2, random_state=KFOLD_SEED)
bc_dt_min_samples_20 = BalanceCascade(estimator=dt_min_samples_20, n_max_subset=10, random_state=KFOLD_SEED)

dt_min_samples_10 = DecisionTreeClassifier(min_samples_leaf=10, max_features=0.2, random_state=KFOLD_SEED)
bc_dt_min_samples_10 = BalanceCascade(estimator=dt_min_samples_10, n_max_subset=10, random_state=KFOLD_SEED)

dt_min_samples_5 = DecisionTreeClassifier(min_samples_leaf=5, max_features=0.2, random_state=KFOLD_SEED)
bc_dt_min_samples_5 = BalanceCascade(estimator=dt_min_samples_5, n_max_subset=10, random_state=KFOLD_SEED)

ensemblers = [bc_dt_min_samples_50, bc_dt_min_samples_20, bc_dt_min_samples_10, bc_dt_min_samples_5]
ensembler_test(logistic_regression, ensemblers)

fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 133422
False Negative = 154
True Positive = 223
False Positive = 66201
f1 score = 0.007
ROC AUC Score = 0.630
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 1
--------test---------
Confusion Matrix:
True Negative = 130890
False Negative = 152
True Positive = 225
False Positive = 68733
f1 score = 0.006
ROC AUC Score = 0.626
fitting sample
(10, 3062, 88) (10, 3062)
training
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Ensembler 2
--------test---------
Confusion Matrix:
True Negative = 131893
False Negative = 133
True Positive = 244
False Positive = 67730
f1 score = 0.007
ROC AUC 

Class weight has to be balanced!
If negative class is heavier, both TPR and FNR decrease, but TPR decrease causes more harm to ROC AUC.
If positive class is heavier, the observation is the opposite

In [33]:
def create_lr_proxy(class_weight='balanced'):
    def create_lr():
        return LogisticRegression(penalty='l2', class_weight=class_weight)
    return create_lr

dt = DecisionTreeClassifier(max_features=0.2, random_state=KFOLD_SEED)
bc = BalanceCascade(estimator=dt, n_max_subset=10, random_state=KFOLD_SEED)

ensembler_test(create_lr_proxy({0: 2, 1: 1}), [bc])
ensembler_test(create_lr_proxy(), [bc])
ensembler_test(create_lr_proxy({0: 1, 1: 2}), [bc])
ensembler_test(create_lr_proxy({0: 1, 1: 4}), [bc])

fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 187970
False Negative = 288
True Positive = 89
False Positive = 11653
f1 score = 0.015
ROC AUC Score = 0.589
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131934
False Negative = 126
True Positive = 251
False Positive = 67689
f1 score = 0.007
ROC AUC Score = 0.663
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 38409
False Negative = 25
True Positive = 352
False Positive = 161214
f1 score = 0.004
ROC AUC Score = 0.563
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 13244
False Negative = 6
True Positive = 371
False Positive = 186379
f1 score = 0.004
ROC AUC Score = 0.525


Try L1 regularization, C=1.0 is just right.

In [36]:
def create_lr_proxy(C=1.0):
    def create_lr():
        return LogisticRegression(penalty='l1', C=C, random_state=KFOLD_SEED)
    return create_lr

dt = DecisionTreeClassifier(max_features=0.2, random_state=KFOLD_SEED)
bc = BalanceCascade(estimator=dt, n_max_subset=10, random_state=KFOLD_SEED)

ensembler_test(create_lr_proxy(2.0), [bc])
ensembler_test(create_lr_proxy(1.0), [bc])
ensembler_test(create_lr_proxy(0.8), [bc])
ensembler_test(create_lr_proxy(0.5), [bc])
ensembler_test(create_lr_proxy(0.2), [bc])

fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131967
False Negative = 125
True Positive = 252
False Positive = 67656
f1 score = 0.007
ROC AUC Score = 0.665
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131859
False Negative = 124
True Positive = 253
False Positive = 67764
f1 score = 0.007
ROC AUC Score = 0.666
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131884
False Negative = 124
True Positive = 253
False Positive = 67739
f1 score = 0.007
ROC AUC Score = 0.666
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Confusion Matrix:
True Negative = 131745
False Negative = 126
True Positive = 251
False Positive = 67878
f1 score = 0.007
ROC AUC Score = 0.663
fitting sample
(10, 3062, 88) (10, 3062)
training
Ensembler 0
--------test---------
Conf